In [1]:
import pandas as pd
import numpy as np
from scipy import stats

csv = pd.read_csv('data/AES-2019.csv')
pd.set_option('use_inf_as_na', True)

In [2]:
# Certain columns are redundant or incorrectly formatted
def organise_dataframe(df):
    df = df[(df['Variable_category'] == 'Financial performance') &
            (df['Variable_name'] == 'Sales of goods and services')]
    df = df[df['Value'].str.contains('C') == False]
    
    df.rename(
        columns={
        'Industry_name_NZSIOC': 'Industry',
        'Value': 'Amount'
        },
        inplace=True
    )
    
    df['Amount'] = df['Amount'].str.replace(',', '').astype(int)
    df.drop_duplicates(subset=['Industry', 'Year'], inplace=True)
    
    df.sort_values(by='Industry', inplace=True)
    df.reset_index(inplace=True)
    
    df.drop(
        columns=[
            'Industry_code_NZSIOC',
            'Industry_aggregation_NZSIOC',
            'Units',
            'Variable_code',
            'Industry_code_ANZSIC06',
            'Variable_category',
            'index'
        ],
        inplace=True
    )
    
    return df

# Some industries have abnormally high growth rates, suggesting inaccurate data
# Those industries are removed here
def remove_outlier_growth_industries(dataframe, outlier_industries):
    return dataframe[~dataframe['Industry'].isin(outlier_industries)]

def calc_overall_growth(first_series, last_series):
    return (last_series - first_series).div(first_series) * 100

In [3]:
datasets_years = [2013, 2014, 2015, 2016, 2017, 2018, 2019]
df_main = organise_dataframe(csv)
dataframes_per_year = []

for i, y in enumerate(datasets_years):
    dataframe = csv[csv['Year'] == y]
    dataframe = organise_dataframe(dataframe)
    if i > 0:
        dataframe = dataframe.assign(Growth = dataframe['Amount'].sub(dataframes_per_year[i-1]['Amount']))
        dataframe = dataframe.assign(Perc_growth = dataframe['Growth'].div(dataframes_per_year[i-1]['Amount']) * 100)
    
    dataframes_per_year.append(dataframe)

In [4]:
outliers_list = set()

for i, df in enumerate(dataframes_per_year):
    if i > 0:
        z_score = stats.zscore(df['Perc_growth'])
        outlier = df[(np.abs(z_score) > 3) | (np.abs(z_score) < -3)]['Industry']
        if len(outlier) > 0:
            outliers_list.add(outlier.iloc[0])

for i, df in enumerate(dataframes_per_year):
    dataframes_per_year[i] = remove_outlier_growth_industries(df, outliers_list)
    
df_main = remove_outlier_growth_industries(df_main, outliers_list)

In [5]:
print('Top 10 Highest Growth Industries (2018-2019):')
dataframes_per_year[-1].sort_values(by='Perc_growth', ascending=False).head(10)

Top 10 Highest Growth Industries (2018-2019):


,Year,Industry,Variable_name,Amount,Growth,Perc_growth
21,2019,Insurance,Sales of goods and services,11136,2321,26.330119
16,2019,Financial and Insurance Services,Sales of goods and services,22836,3180,16.178266
15,2019,Financial Asset Investing,Sales of goods and services,2424,281,13.112459
8,2019,Computer System Design and Related Services,Sales of goods and services,9903,1048,11.835121
1,2019,"Advertising, Market Research and Management Se...",Sales of goods and services,11227,1122,11.103414
6,2019,"Building Cleaning, Pest Control and Other Supp...",Sales of goods and services,4211,414,10.903345
43,2019,Travel Agency and Tour Arrangement Services,Sales of goods and services,1142,112,10.873786
10,2019,Construction Services,Sales of goods and services,27364,2612,10.552683
36,2019,Rental and Hiring Services (except Real Estate),Sales of goods and services,6483,606,10.311383
11,2019,Electricity and Gas Supply,Sales of goods and services,18000,1666,10.199584


In [6]:
df_growth_overall = pd.DataFrame()
df_growth_overall = df_growth_overall.assign(Industry=dataframes_per_year[-1]['Industry'])
df_growth_overall = df_growth_overall.assign(
    Overall_growth=calc_overall_growth(
        dataframes_per_year[0]['Amount'],
        dataframes_per_year[-1]['Amount'])
)

In [7]:
# For use in Tableau
df_main.to_csv('data/main_data.csv')
df_growth_overall.to_csv('data/growth_overall.csv')